# Get Collateral

In this notebook, we compile the data for collateral change transactions from the API.

In [1]:
#import packages
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import matplotlib.dates as md
import math
import time

In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_tes

## Fetch Collateral Change Transaction Data

First, if this data was previously compiled, we load the existing data.

In [2]:
start_time = 0
try:
    old_data = pd.read_csv('collaterals.csv')
    exists = True
    start_time = old_data['timestamp'].max()
    old_data.info()
except:
    old_data=pd.DataFrame()
    exists = False

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226423 entries, 0 to 226422
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   fromState  226423 non-null  bool  
 1   pool       226423 non-null  object
 2   reserve    226423 non-null  object
 3   timestamp  226423 non-null  int64 
 4   toState    226423 non-null  bool  
 5   user       226423 non-null  object
 6   type       226423 non-null  object
dtypes: bool(2), int64(1), object(4)
memory usage: 9.1+ MB


Here, we write a query to fetch transaction information from the API.

In [3]:
lastId='""'
collateral_data=[]
#loop until no more data left
while(1):
    try:
        #set query
        query="""
        {
            usageAsCollaterals (first: 1000 orderBy: id where:{id_gt:"""+lastId+""", timestamp_gt:"""+str(start_time)+"""}) {
            id
            user{
                id
            },
            pool{
                id
            },
            reserve {
              id,
              symbol
            },
            fromState,
            toState,
            timestamp
            }
        }
        """
        #make request
        url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
        request = requests.post(url,json={'query':query})
        #store data
        collateral_data.extend(request.json()['data']['usageAsCollaterals'])
        lastId = "\""+request.json()['data']['usageAsCollaterals'][-1]['id']+"\""
    except:
        #exit when no more data left to get
        break

#create borrows data frame
df_collaterals = pd.DataFrame(collateral_data)
df_collaterals['type']='collateral'
df_collaterals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6777 entries, 0 to 6776
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         6777 non-null   object
 1   user       6777 non-null   object
 2   pool       6777 non-null   object
 3   reserve    6777 non-null   object
 4   fromState  6777 non-null   bool  
 5   toState    6777 non-null   bool  
 6   timestamp  6777 non-null   int64 
 7   type       6777 non-null   object
dtypes: bool(2), int64(1), object(5)
memory usage: 331.0+ KB


### Re-Format Collateral Change Transaction Data

Some of the values in the dataset are dictionaries, so we must re-format the data to get the wanted field from those dictionaries.

In [4]:
#get id's:
def getUser(row):
    return row['user']['id']
df_collaterals['user']=df_collaterals.apply(lambda x: getUser(x), axis=1)

def getPool(row):
    return row['pool']['id']
df_collaterals['pool']=df_collaterals.apply(lambda x: getPool(x), axis=1)

#get symbols
def getReserve(row):
    if not isinstance(row['reserve'],float):
        return row['reserve']['symbol']
    else:
        return np.nan
df_collaterals['reserve']=df_collaterals.apply(lambda x: getReserve(x), axis=1)

In [5]:
#reset index
df_collaterals = df_collaterals.set_index('id')        

### Save Final Data Frame

Finally, we save the dataframe to a csv file.

In [6]:
df_collaterals = old_data.append(df_collaterals)
df_collaterals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 233200 entries, 0 to 0xfffb7da62057ec772005d89fae8aa5fc6455dd4e0abc7fa075a74005b72f5d51:6
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   fromState  233200 non-null  bool  
 1   pool       233200 non-null  object
 2   reserve    233200 non-null  object
 3   timestamp  233200 non-null  int64 
 4   toState    233200 non-null  bool  
 5   user       233200 non-null  object
 6   type       233200 non-null  object
dtypes: bool(2), int64(1), object(4)
memory usage: 11.1+ MB


In [7]:
df_collaterals.to_csv("collaterals.csv",index=False)